### Common imports

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Output, Dropdown, SelectMultiple, HBox, VBox, Button, IntSlider, FloatRangeSlider, FloatSlider, Text, Textarea, Combobox, HBox, VBox, SelectMultiple, Tab
from IPython.display import display

from surianalytics.connectors import RESTSciriusConnector
from surianalytics.datamining import min_max_scaling

import pandas as pd
import numpy as np

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', None)

In [ ]:
c = RESTSciriusConnector()
c.set_page_size(1000)

### Timestamp handlers

In [ ]:
from datetime import datetime, timedelta

In [ ]:
TIME_END = datetime.utcnow()
TIME_BEGINNING = TIME_END - timedelta(minutes=60)

In [ ]:
SLIDER_TIME_MINUTES = IntSlider(
    value=60,
    min=5,
    max=600,
    step=5,
    description='Minutes',
    orientation='horizontal',
    readout=True,
)

In [ ]:
# TimePicker is not in ipywidgests 7.7
# Use text boxes instead for now
TEXT_TIME_BEGINNING = Text(
    description="From: ",
    value=TIME_BEGINNING.isoformat(),
    continuous_update=True
)
TEXT_TIME_END = Text(
    description="To: ",
    value=TIME_END.isoformat(),
    continuous_update=True
)
_ = c.set_query_timeframe(from_date=TEXT_TIME_BEGINNING.value, to_date=TEXT_TIME_END.value)

In [ ]:
def handler_update_timeframe(args):
    c.set_query_timeframe(from_date=TEXT_TIME_BEGINNING.value, to_date=TEXT_TIME_END.value)

In [ ]:
BUTTON_UPDATE_TIME = Button(description="Set time")
BUTTON_UPDATE_TIME.on_click(handler_update_timeframe)

In [ ]:
def handler_reset_timeframe(args):
    time_to = datetime.utcnow()
    time_from = time_to - timedelta(minutes=SLIDER_TIME_MINUTES.value)
    TEXT_TIME_END.value = time_to.isoformat()
    TEXT_TIME_BEGINNING.value = time_from.isoformat()
    c.set_query_delta(minutes=SLIDER_TIME_MINUTES.value)

In [ ]:
BUTTON_RESET_TIME = Button(description="Generate time")
BUTTON_RESET_TIME.on_click(handler_reset_timeframe)

In [ ]:
import pickle
PKL_TIME = "./time.pkl"

In [ ]:
def handler_pickle_time_save(args):
    with open(PKL_TIME, "wb") as handle:
        pickle.dump((TEXT_TIME_BEGINNING.value, TEXT_TIME_END.value), handle)

In [ ]:
BUTTON_SAVE_TIME = Button(description="Dump time")
BUTTON_SAVE_TIME.on_click(handler_pickle_time_save)

In [ ]:
def handler_pickle_time_load(args):
    handle = open(PKL_TIME, "rb")
    times = pickle.load(handle)
    TEXT_TIME_BEGINNING.value = times[0]
    TEXT_TIME_END.value = times[1]
    handler_update_timeframe(args)

In [ ]:
BUTTON_LOAD_TIME = Button(description="Sync time")
BUTTON_LOAD_TIME.on_click(handler_pickle_time_load)

In [ ]:
BOX_TIME = VBox([
    HBox([SLIDER_TIME_MINUTES, BUTTON_RESET_TIME]), 
    HBox([
        VBox([TEXT_TIME_BEGINNING, TEXT_TIME_END]), 
        BUTTON_UPDATE_TIME,
        VBox([BUTTON_SAVE_TIME, BUTTON_LOAD_TIME])
    ])
])

In [ ]:
BOX_TIME

### Query handler

In [ ]:
TEXT_QUERY = Textarea(
    description='Query filter:',
    value="*",
    continuous_update=True
)

In [ ]:
import pickle
PKL_QUERY = "./query.pkl"

In [ ]:
def handler_pickle_query_save(args):
    with open(PKL_QUERY, "wb") as handle:
        pickle.dump(TEXT_QUERY.value, handle)

In [ ]:
BUTTON_SAVE_QUERY = Button(description="Dump query")
BUTTON_SAVE_QUERY.on_click(handler_pickle_query_save)

In [ ]:
def handler_pickle_query_load(args):
    handle = open(PKL_QUERY, "rb")
    TEXT_QUERY.value = pickle.load(handle)

In [ ]:
BUTTON_LOAD_QUERY = Button(description="Sync query")
BUTTON_LOAD_QUERY.on_click(handler_pickle_query_load)

In [ ]:
BOX_QUERY = HBox([
    TEXT_QUERY, 
    VBox([BUTTON_SAVE_QUERY, BUTTON_LOAD_QUERY])
])

### Graph Mining

In [ ]:
if "FIELDS" not in globals():
    FIELDS = []
if "GRAPH" not in globals():
    GRAPH = None
    
GRAPH_WIDTH = 1600
GRAPH_HEIGHT = 900

In [ ]:
DROPDOWN_EVENT_TYPES = Dropdown(
    options=["all"],
    value="all",
    description="Event type"
)
DROPDOWN_FIELDS_SRC = Combobox(
    options=FIELDS,
    value=None
)
DROPDOWN_FIELDS_DEST = Combobox(
    options=FIELDS,
    value=None
)
SLIDER_SIZE_COL = IntSlider(
    value=500,
    min=100,
    max=5000,
    description='Aggregation max size',
    orientation='horizontal',
    readout=True,
)
SLIDER_THRESH_DEGREE = FloatRangeSlider(
    value=[0, 1],
    min=0,
    max=1,
    step=0.001,
    description="Degree: ",
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [ ]:
def handler_update_event_types(args):
    DROPDOWN_EVENT_TYPES.options = c.get_event_types()

In [ ]:
BUTTON_REFRESH_EVENT_TYPES = Button(description="Refresh event types")
BUTTON_REFRESH_EVENT_TYPES.on_click(handler_update_event_types)

In [ ]:
FIELDS_FILTER = ["@timestamp", "timestamp"]
def handler_build_fields(event_type="all"):
    fields = c.get_unique_fields(event_type=DROPDOWN_EVENT_TYPES.value)
    if len(fields) == 0:
        print("no fields from", event_type)
        
    global FIELDS
    FIELDS = [f for f in fields if f not in FIELDS_FILTER]
    DROPDOWN_FIELDS_SRC.options = FIELDS
    DROPDOWN_FIELDS_DEST.options = FIELDS
    
_ = interactive(handler_build_fields, event_type=DROPDOWN_EVENT_TYPES)

In [ ]:
OUTPUT_GRAPH_DL = Output()
def handler_pull_graph_data(args):
    OUTPUT_GRAPH_DL.clear_output()
    with OUTPUT_GRAPH_DL:
        display(print("Calling scirius"))
        global GRAPH
        GRAPH = c.get_eve_fields_graph_nx(        
            col_src=DROPDOWN_FIELDS_SRC.value,
            col_dest=DROPDOWN_FIELDS_DEST.value,
            size_src=SLIDER_SIZE_COL.value,
            size_dest=SLIDER_SIZE_COL.value,
            event_type=DROPDOWN_EVENT_TYPES.value,
            qfilter=TEXT_QUERY.value
        )
        display("call done, got {} nodes and {} edges".format(len(GRAPH.nodes()), len(GRAPH.edges())))

In [ ]:
BUTTON_LOAD_DATA = Button(description="Pull data")
BUTTON_LOAD_DATA.on_click(handler_pull_graph_data)

In [ ]:
import networkx as nx
import hvplot.networkx as hvnx
import holoviews as hv
from holoviews import opts
from surianalytics.datamining import min_max_scaling

In [ ]:
OUTPUT_GRAPH = Output()
def handler_draw_graph(args):
    OUTPUT_GRAPH.clear_output()
    with OUTPUT_GRAPH:
        if len(GRAPH.edges) == 0:
            display("Missing graph data, please run query first")
            return

        # use local graph object
        g = GRAPH.copy()

        # drop empty nodes (and connected edges) 
        # means missing eve field, no connection can be made
        if "" in list(g.nodes()):
            g.remove_node("")

        doc_counts = [attr["doc_count"] for (src, dst, attr) in g.edges(data=True)]
        doc_counts = np.log2(doc_counts)
        doc_counts = min_max_scaling(pd.Series(doc_counts))

        # add scaled doc counts to edges to serve as weights
        for i, (src, dst, attr) in enumerate(g.edges(data=True)):
            attr["scaled_doc_count"] = doc_counts[i]

        # discover node degree and scale the values
        degree = [g.degree(n) for n in g.nodes()]
        degree = min_max_scaling(pd.Series(degree))

        to_remove = []
        # drop nodes that do not match filtering criteria
        for i, n in enumerate(g.nodes()):
            if degree[i] > SLIDER_THRESH_DEGREE.value[1] or degree[i] < SLIDER_THRESH_DEGREE.value[0]:
                to_remove.append(n)

        for n in to_remove:
            g.remove_node(n)

        # calculate total number of docs for edges
        doc_count_total_edge = np.sum([attr["doc_count"] for (src, dst, attr) in g.edges(data=True)])

        # locate source nodes
        n_src = [i for i, (u, a) in enumerate(g.nodes(data=True)) if a["kind"] == "source"]
        # locate destination nodes
        n_dst = [i for i, (u, a) in enumerate(g.nodes(data=True)) if a["kind"] == "destination"]

        # generate layout
        pos = nx.layout.spring_layout(g)

        # generate nodes per kind
        nodes_src = hvnx.draw_networkx_nodes(g, pos, nodelist=n_src, node_color='#A0CBE2').opts(width=GRAPH_WIDTH, height=GRAPH_HEIGHT)
        nodes_dst = hvnx.draw_networkx_nodes(g, pos, nodelist=n_dst, node_color="Orange").opts(width=GRAPH_WIDTH, height=GRAPH_HEIGHT)

        # use kwargs to make parameter handling easier
        edge_params = {
            "alpha": 1,
            "edge_color": 'scaled_doc_count', 
            "edge_cmap": 'viridis',
            "edge_width": hv.dim('scaled_doc_count')*5
        }

        # generate edges
        edges = hvnx.draw_networkx_edges(g, pos, **edge_params).opts(width=GRAPH_WIDTH, height=GRAPH_HEIGHT)

        # overlay nodes and edges
        res = edges * nodes_src * nodes_dst

        component_sizes = [len(c) for c in sorted(nx.connected_components(g), key=len, reverse=True) if len(c) > 1]

        display(print("Number of documents for edges: {}".format(doc_count_total_edge)))
        display(print("Number of clusters: {}".format(len(component_sizes))))
        display(res)

In [ ]:
BUTTON_DRAW_GRAPH = Button(description="Draw graph")
BUTTON_DRAW_GRAPH.on_click(handler_draw_graph)

In [ ]:
BOX_COL_SRC = HBox([DROPDOWN_FIELDS_SRC, SLIDER_THRESH_DEGREE])
BOX_COL_DEST = HBox([DROPDOWN_FIELDS_DEST])

BOX_COL_CONFIG = VBox([BOX_COL_SRC, BOX_COL_DEST])
BOX_BUTTONS = HBox([BUTTON_REFRESH_EVENT_TYPES, BUTTON_LOAD_DATA, BUTTON_DRAW_GRAPH])

BOX_TIME_INPUT = VBox([TEXT_TIME_BEGINNING, TEXT_TIME_END])
BOX_TIME_BTNS = VBox([BUTTON_UPDATE_TIME, BUTTON_RESET_TIME])

BOX_FINAL = VBox([
    DROPDOWN_EVENT_TYPES, 
    BOX_COL_CONFIG, 
    BOX_BUTTONS,
    OUTPUT_GRAPH_DL,
    OUTPUT_GRAPH
])

### Hunting section

In [ ]:
tab = Tab(children=[BOX_TIME, BOX_QUERY, BOX_FINAL])
tab.set_title(0, "Timepicker")
tab.set_title(1, "Query filter")
tab.set_title(2, "Graph")

In [ ]:
tab